In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

## 2.3 Negotiations, auctions, and optimization
### 2.3.1 From contract nets to auction-like optimization
In a contract net we have a global problem, made up of a number of tasks. For each agent there is a cost function $c_i(T)$ reflecting the cost of agent $i$ to do all the tasks $T$. We start agents out with random tasks, and want them to swap and exchange them in order to lower the sum of the individual costs. In future chapters we will look at how to do this with agents which are also self-interested. First, we derive a linear progrmaming solution to this problem. 
### 2.3.2 The assignment problem and linear programming
In a (symmetric) assignment problem we have a set $N$ of agents, a set $X$ of objects, a set of possible assignment pairs $M$, and a function $v$ which tells us what the value of an assignment pair is. At the moment we just consider that we have at most 1 object per agent. A valid assignment is then a set of pairs $S\subseteq M$, which has the property that each each agent $i$ and object $j$ is in at most 1 pair. A feasable assignment is one in which all agents are assigned an object, and an optimal assignment is one in which the sum of the values is maximised. 

This problem can be expressed as a linear program. We introduce an indicator matrix $x$ where $x_{i,j}=1$ if the pair ($i$,$j$) is selected. Then the linear program is:

$$
\begin{align*}
\text{maximise} \quad & \sum_{i,j\in M} v(i,j)x_{i,j} \\
\text{subject to} \quad & \sum_{j} x_{i,j}<=1 \\
\text{} \quad & \sum_{i} x_{i,j}<=1
\end{align*}
$$

While this might seem incorrect, as this would seem to possibly create partial assignments, in truth the answers turn out to be integers. In fact, every linear program encoding of an assignment problem has an integer solution. Solving a linear program can be done in polynomial time, using the simplex algorithm.

In [30]:
agents = ["john","mary","leon"]
objects = ["cat","dog","hat"]
value_dict = {
    ("john","cat"):1,("john","dog"):3,("john","hat"):1,
    ("mary","cat"):5,("mary","dog"):1,("mary","hat"):0,
    ("leon","cat"):2,("leon","dog"):2,("leon","hat"):3,
}
value_vector = list(value_dict.values())
x = np.zeros(9)
A = np.array([
    [1,1,1,0,0,0,0,0,0], # john can't be assigned to more than 1 item.
    [0,0,0,1,1,1,0,0,0], # mary
    [0,0,0,0,0,0,1,1,1], # leon
    [1,0,0,1,0,0,1,0,0], # the cat can't be assigned to more than 1 person
    [0,1,0,0,1,0,0,1,0], # the dog
    [0,0,1,0,0,1,0,0,1], # the hat
])
b = np.ones(6)
res = linprog(-np.array(value_vector), A_ub=A, b_ub=b, bounds=[(0,None) for _ in range(9)])
print("resulting weights:")
print(res["x"])
print("assignment")
assignment = [list(value_dict.keys())[i] for i in np.where(res["x"]==1)[0]]
print(assignment)

resulting weights:
[0. 1. 0. 1. 0. 0. 0. 0. 1.]
assignment
[('john', 'dog'), ('mary', 'cat'), ('leon', 'hat')]


the problem with this is that it isn't particularly parallelizable, and that small changes in the inputs mean the problem has to be redone from scratch. Here we explore an alternative, based on the idea of competitive equilibrium:

#### The assignment problem and competitive equilibrium

Imagine that the goods in $S$ have a price $p$, and lets define the utility of an agent as $u(i,j)=v(i,j)-p_j$.

An assignment and a set of prices are in *competitive equilibrium* if each agent is assigned an object which maximises their utility. I.e., every $u(i,j)>u(i,k)$.

While this seems odd, as we don't really have the concept of prices, it turns out that there is a nice theorem about competitive equilibrium:

***Theorem***

If a feasible assignment and a price vector are in competitive equilibrium, then $S$ is an optimal assignment. At the same time, for any optimal solution $S$, there is a corresponding price vector such that the two satisfy the competitive equilibrium property.

#### competitive equilibrium and primal-dual problems






What this theorem means is that one way to search the space of possible assignments is to instead have the agents 'bid' for the goods.